# Comparison Setup, loads TSPs and creates pandas dataframe

## Imports

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import tsplib95

from wsp import tsp, ds, util # TODO: refine imports

TREE_TYPE = ds.PKPRQuadTree
BF_THRESHOLD = 14

## Loading Euclidean TSPs

In [ ]:
problems : list[tsplib95.models.StandardProblem] = []

for file in os.listdir("ALL_tsp"): # Loop through every tsp
    if not file.endswith(".tsp"):
        continue
    problem = tsplib95.load(f"ALL_tsp/{file}")
    if problem.edge_weight_type != "EUC_2D": # Skip non-Euclidean TSPs
        continue # TODO: include ATT and GEO and maybe EUC_3D
    
    if problem.name in ["a280", "rd100"]: # Special skip cases, TODO: remove
        continue
    
    problems.append(problem)
    print(f"Added {problem.name}")

print("Found", len(problems), "euclidean TSPs")

## Setting up data science

In [ ]:
# Create a data frame with columns for the name opt solution, nearest neighbor solution, and untouched solution

if os.path.exists("results.pkl"):
    df = pd.read_pickle("results.pkl")
else:
    df = pd.DataFrame(columns=['name', 'num_points', 'opt_len', 'nn_len', 'untouched_len', 'nwsp5_len', 'nwsp7_len', 'nwsp9_len'])
    df.name = pd.Series(sorted([problem.name for problem in problems]))
    df.set_index("name", inplace = True)
df

## Save Default Values into dataframe

In [ ]:

# fig, ax = plt.subplots(1, 2, figsize=(12,6))
ax = None
i = 0
for problem in problems:
    print(i, problem.name)
    # ax[0].clear()
    # ax[1].clear()

    points = [ds.Point(*problem.node_coords[i]) for i in problem.get_nodes()]

    ts_problem = tsp.TravellingSalesmanProblem[TREE_TYPE](TREE_TYPE, points, ax, s=1.0)
    
    # MARK: load the dp solution
    num_path = None
    if os.path.exists(f"ALL_tsp/{problem.name}.opt.tour"): # If there is an optimal tour file
        num_path = tsplib95.load(f"ALL_tsp/{problem.name}.opt.tour").tours[0]
    elif len(problem.tours) > 0:
        num_path = problem.tours[0] # NOTE: I do not know of any euclidean which has this property
        raise ValueError("A shooting star")
    if num_path is not None:
        path = [ts_problem.points[i-1] for i in num_path]
        ts_problem.dp_path = (path + [path[0]], util.calc_dist(path), None)
        # ts_problem.draw_path(ts_problem.dp_path[0], '#FFC0CB')
    elif len(ts_problem.points) <= BF_THRESHOLD:
        print("Brute forcing", problem.name)
        ts_problem.dp_path # Load the property

    # MARK: Populate row values
    df.loc[problem.name, "num_points"] = len(ts_problem.points)
    
    if "dp_path" in ts_problem.__dict__:
        assert ts_problem.check_tour(ts_problem.dp_path[0]), "Invalid path"
        df.loc[problem.name, "opt_len"] = ts_problem.dp_path[1]
    if pd.isna(df.loc[problem.name, "nn_len"]):
        assert ts_problem.check_tour(ts_problem.nnn_path[0]), "Invalid path"
        df.loc[problem.name, "nn_len"] = ts_problem.nnn_path[1]
    if pd.isna(df.loc[problem.name, "untouched_len"]):
        assert ts_problem.check_tour(ts_problem.untouched_path[0]), "Invalid path"
        df.loc[problem.name, "untouched_len"] = ts_problem.untouched_path[1]
    # if pd.isna(df.loc[problem.name, "nwsp5_len"]):
    #     assert ts_problem.check_tour(ts_problem.nwsp_path(5)[0]), "Invalid path"
    #     df.loc[problem.name, "nwsp5_len"] = ts_problem.nwsp_path(5)[1]
    # if pd.isna(df.loc[problem.name, "nwsp7_len"]):
    #     assert ts_problem.check_tour(ts_problem.nwsp_path(7)[0]), "Invalid path"
    #     df.loc[problem.name, "nwsp7_len"] = ts_problem.nwsp_path(7)[1]
    # if df.loc[problem.name, "nwsp9_len"]:
    #     df.loc[problem.name, "nwsp9_len"] = ts_problem.nwsp_path(9)[1]
    
    i += 1
print("done") # TODO: use progress instead of prints

In [ ]:
df #[df['opt_len'] > 0]

In [ ]:
df.to_pickle('results.pkl')